In [ ]:
#reload the modules before running the program
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy
from scipy import signal
from scipy.fft import fftshift
from scipy.fft import fft, fftfreq

import matplotlib.pyplot as plt
import matplotlib as mpl

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import os
import math
import cv2

In [ ]:
from datasets import RFData, CameraData, SignalDataset, FuseDatasets
import metrics
import utils

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
data_path = "E:\OSA_project\OSA_project\Datasets\osa_new_processed"
# trial_folders = ["v_12_5", "v_13_5", "v_13_10"]
trial_folders = os.listdir(data_path)

train_idxs = np.linspace(0, 49, 50, dtype=int)
train_folders = [trial_folders[i] for i in train_idxs]
test_folders = [trial_folders[i] for i in range(len(trial_folders)) if i not in train_idxs]


thermal_file_name = "Thermal_Camera"
num_samps_oversample = 20 # per experiment, number of samples to generate
data_length = 9000
fs = 30
out_len = 1800 # sample length generated
thermal_ext = ".tiff"

dataset_thermal_train = CameraData(data_path, train_folders, thermal_file_name, num_samps_oversample, fs, data_length, out_len, thermal_ext)
dataset_thermal_test = CameraData(data_path, test_folders, thermal_file_name, num_samps_oversample, fs, data_length, out_len, thermal_ext)

samp_f=5e6
freq_slope=60.012e12
samples_per_chirp=256
num_tx = 3
num_rx = 4
radar_file_name = "FMCW_Radar.npy"
window_size = 5 # number of range bins to use

dataset_radar_train = RFData(data_path, train_folders, data_length, radar_file_name, out_len, window_size, samples_per_chirp, samp_f, freq_slope, num_samps_oversample, num_tx, num_rx, fs)
dataset_radar_test = RFData(data_path, test_folders, data_length, radar_file_name, out_len, window_size, samples_per_chirp, samp_f, freq_slope, num_samps_oversample, num_tx, num_rx, fs)

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
vital_dict_file_name = "gt_dict.pkl"
vital_key_radar = "CHEST"
vital_key_thermal = "AIR_flow"
l_freq_bpm = 5
u_freq_bpm = 30
dataset_OSA_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'OSA', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
                              normalize=False)
dataset_CSA_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'CSA', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
                              normalize=False)
dataset_hypopnea_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'Hypopnea', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
                              normalize=False)
dataset_partial_apnea_train = SignalDataset(data_path, train_folders, vital_dict_file_name, 'Partial_Apnea', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
                              normalize=False)
dataset_gt_thermal_train = SignalDataset(data_path, train_folders, vital_dict_file_name, vital_key_thermal, data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)
dataset_gt_radar_train = SignalDataset(data_path, train_folders, vital_dict_file_name, vital_key_radar, data_length, out_len, True, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)

# dataset_apnea_test = SignalDataset(data_path, test_folders, vital_dict_file_name, 'Sleep_Apnea', data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False,
#                               normalize=False)
# dataset_gt_thermal_test = SignalDataset(data_path, test_folders, vital_dict_file_name, vital_key_thermal, data_length, out_len, False, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)
# dataset_gt_radar_test = SignalDataset(data_path, test_folders, vital_dict_file_name, vital_key_radar, data_length, out_len, True, fs, 1024, False, l_freq_bpm, u_freq_bpm, num_samps_oversample, False)

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
fused_dataset_train = FuseDatasets([dataset_radar_train, dataset_thermal_train, dataset_gt_radar_train, dataset_gt_thermal_train, dataset_apnea_train, dataset_hypopnea_train, dataset_partial_apnea_train], ["radar", "thermal", "gt_radar", "gt_ir", "gt_apnea", "hypopnea", "partial_apnea"], out_len=out_len)
# fused_dataset_test = FuseDatasets([dataset_radar_test, dataset_thermal_test, dataset_gt_radar_test, dataset_gt_thermal_test, dataset_apnea_test], ["radar", "thermal", "gt_radar", "gt_ir", "gt_apnea"], out_len=out_len)

In [9]:
save_path = r"E:\OSA_project\Emir_Clean\pre_load_datav2"

gt_OSA = np.load(os.path.join(save_path,r"gt_OSA.npy"))
gt_CSA  = np.load(os.path.join(save_path,r"gt_CSA.npy"))
thermal_arr = np.load(os.path.join(save_path,r"thermal_arr.npy"))
# np.load(os.path.join(save_path,r"thermal_vid_arr.npy")_vid_arr)
gt_hypopnea = np.load(os.path.join(save_path,r"gt_hypopnea.npy"))
radar_arr = np.load(os.path.join(save_path,r"radar_arr.npy"))
gt_radar = np.load(os.path.join(save_path,r"gt_radar_arr.npy"))
gt_ir = np.load(os.path.join(save_path,r"gt_ir_arr.npy"))

In [11]:
apnea_idx = []
folds = []
count = 0
for i in range(len(gt_OSA)):
    # print(i)
    try:
        # folds.append(gt_OSA.trial_folders[gt_OSA.oversampling_idxs[i][0]])
        batch = gt_OSA[i]
        if(np.mean(batch) > 0.1):
            apnea_idx.append(i)
    except:
        # print(gt_OSA.trial_folders[dataset_OSA_train.oversampling_idxs[i][0]])
        count = count + 1
        pass

print(100*len(apnea_idx)/len(gt_OSA))

0.8269230769230769
0


In [ ]:
NUKS_idx = np.load(r"K:\OSA_project\Kai_data\NUKS_idx.npy")
mode_lock_idx = np.load(r"K:\OSA_project\Kai_data\mode_lock_idx.npy")
movement_idx = np.load(r"K:\OSA_project\Kai_data\combined_movement_idx.npy")

In [ ]:
gt_apnea = np.load(r"K:\OSA_project\Emir_Clean\pre_load_data\gt_apnea.npy")
gt_hypopnea = np.load(r"K:\OSA_project\Emir_Clean\pre_load_data\gt_hypopnea.npy")
gt_ir = np.load(r"K:\OSA_project\Emir_Clean\pre_load_data\gt_ir_arr.npy")
gt_radar = np.load(r"K:\OSA_project\Emir_Clean\pre_load_data\gt_radar_arr.npy")
radar_arr = np.load(r"K:\OSA_project\Emir_Clean\pre_load_data\radar_arr.npy")
thermal_arr = np.load(r"K:\OSA_project\Emir_Clean\pre_load_data\thermal_arr.npy")
# thermal_vid_arr = np.load(r"E:\OSA_project\Emir_Clean\thermal_vid_arr.npy")

In [ ]:
t_arr = np.linspace(0, 1800/30, 1800)

In [ ]:
def predict(thermal, dmin, dmax, th, mode='max', percentage=25, plot=False):
    """
    Input: 
    thermal: 1d-array, thermal signal
    dmin, dmax: int, optional, size of chunks
    th: float, threshold value
    mode: str, optional, mode of thresholding
    Output:
    pred: 1d-array, binary prediction
    """
    t_arr = np.linspace(0, len(thermal)/30, len(thermal))
    lmin, lmax = utils.hl_envelopes_idx(thermal, dmin=dmin, dmax=dmax)
    max_th = np.interp(t_arr, t_arr[lmax], thermal[lmax])
    min_th = np.interp(t_arr, t_arr[lmin], thermal[lmin])

    if(plot == True):
        plt.plot(t_arr, thermal, color='black', label='Thermal Image Signal')
        utils.plot_envelope(t_arr, thermal, lmin, lmax)
    pred = None
    if(mode == 'max'):
        pred = ((max_th - min_th)/max(max_th - min_th) < th).astype(int)
    elif(mode == '90th'):
        pred = ((max_th - min_th)/np.percentile(max_th - min_th, 90) < th).astype(int)
    elif(mode == 'median'):
        pred = ((max_th - min_th)/np.median(max_th - min_th) < th).astype(int)
    elif(mode == 'mean'):
        pred = ((max_th - min_th)/np.mean(max_th - min_th) < th).astype(int)
    else:
        pred = ((max_th - min_th)/np.percentile(max_th - min_th, percentage) < th).astype(int)
    return pred, max_th, min_th

In [ ]:
def find_movement_peaks(dists_min, dists_max, t_arr, th, x_th, lmin, lmax):
    t_arr_min = t_arr[lmin]
    idx_min = []
    x_min = []
    y_min = []

    for i in range(len(dists_min)):
        if(dists_min[i] > th):
            idx_min.append(lmin[i])
            x_min.append(t_arr_min[i])
            y_min.append(dists_min[i])
    
    center_list_min = []
    iter = 0
    while((len(x_min) != 0) and (iter < len(y_min))):
        offset = 1
        avg_pos_arr = [(x_min[iter], y_min[iter], idx_min[iter])]
        while(((iter + offset) < len(y_min)) and (abs(x_min[iter] - x_min[iter+offset]) < x_th)):
            avg_pos_arr.append((x_min[iter+offset], y_min[iter+offset], idx_min[iter+offset]))
            offset += 1
        
        centers = []
        for xy in avg_pos_arr:
            x_min.remove(xy[0])
            y_min.remove(xy[1])
            idx_min.remove(xy[2])
            centers.append(xy[2])
        center_list_min.append(np.median(centers)) # can change this to be the mean

    t_arr_max = t_arr[lmax]
    idx_max = []
    x_max = []
    y_max = []

    for i in range(len(dists_max)):
        if(dists_max[i] > th):
            idx_max.append(lmax[i])
            x_max.append(t_arr_max[i])
            y_max.append(dists_max[i])
    
    center_list_max = []
    iter = 0
    while((len(x_max) != 0) and (iter < len(y_max))):
        offset = 1
        avg_pos_arr = [(x_max[iter], y_max[iter], idx_max[iter])]
        while(((iter + offset) < len(y_max)) and (abs(x_max[iter] - x_max[iter+offset]) < x_th)):
            avg_pos_arr.append((x_max[iter+offset], y_max[iter+offset], idx_max[iter+offset]))
            offset += 1
        
        centers = []
        for xy in avg_pos_arr:
            x_max.remove(xy[0])
            y_max.remove(xy[1])
            idx_max.remove(xy[2])
            centers.append(xy[2])
        center_list_max.append(np.median(centers)) # can change this to be the mean
    # print("Center List Max: ", center_list_max, '\n')
    # print("Center List Min: ", center_list_min, '\n')
    for i in range(len(center_list_max)):
        t = t_arr[int(center_list_max[i])]
        for j in range(len(center_list_min)):
            # print("t: ", t, "t_arr: ", t_arr[int(center_list_min[j])], '\n')
            # print(abs(t - t_arr[int(center_list_min[j])]), x_th, '\n')
            if((len(center_list_min) != 0) and (abs(t - t_arr[int(center_list_min[j])]) < x_th/30)):
                center_list_min.remove(center_list_min[j])
                break
    center_list_max.extend(center_list_min)

    return center_list_max
    
def remove_peaks(center_list, sig, half_length=90):
    """
    Input: 
    center_list: list, indices of peaks
    sig: 1d-array, signal
    Output:
    sig: 1d-array, signal with peaks removed
    """
    signal_comps = []
    last_lower = -1
    for i in range(len(center_list)):
        cut_center = int(center_list[i])
        lower = max(0, cut_center-half_length)
        upper = min(len(sig), cut_center+half_length)
        # print("lower: ", lower, "upper: ", upper, '\n')
        if(i == 0):
            signal_comps.append([sig[:lower], [0, lower]])
            last_upper = upper
        if((i != (len(center_list) - 1)) and (i != 0)):
            signal_comps.append([sig[last_upper:lower], [last_upper, lower]])
            last_upper = upper
        if(i == (len(center_list) - 1)):
            signal_comps.append([sig[last_upper:lower], [last_upper, lower]])
            signal_comps.append([sig[upper:], [upper, len(sig)]])
    return signal_comps

In [ ]:
def movement_detector(thermal, dmin=29, dmax=31, th=4.5, plot=False, include_edges=False):

    lmin, lmax = utils.hl_envelopes_idx(thermal, dmin=dmin, dmax=dmax)

    min_env = thermal[lmin]
    max_env = thermal[lmax]
    half_len = 4 

    dists_min = []
    dists_max = []

    if(include_edges == True):
        start_vec = [(max_env[0]-max_env[1+i])**2 for i in range(half_len)]
        start_vec = np.array(start_vec)
        dists_max.append(np.sum(start_vec)/((1 or np.mean(start_vec[np.argsort(start_vec)[::-1]]))*len(start_vec)))

   
    for i in range(1, len(max_env)-1):
        vec_start = max(0, i - half_len)
        vec_end = min(len(max_env), i + half_len)
        vec = max_env[vec_start:vec_end]
        vec = np.array(vec)
        center_vec = np.repeat(max_env[i], len(vec))
        dist = np.sum((center_vec - vec)**2)/((1 or np.mean(vec[np.argsort(vec)[::-1]]))*len(vec))
        dists_max.append(dist)

    if(include_edges == True):
        end_vec = [(max_env[len(max_env)-1]-max_env[len(max_env)-2-i])**2  for i in range(half_len)]
        end_vec = np.array(end_vec)
        dists_max.append(np.sum(end_vec)/((1 or np.mean(end_vec[np.argsort(end_vec)[::-1]]))*len(end_vec)))

        start_vec = [(min_env[0]-min_env[1+i])**2 for i in range(half_len)]
        start_vec= np.array(start_vec)
        dists_min.append(np.sum(start_vec)/((1 or np.mean(start_vec[np.argsort(start_vec)[::-1]]))*len(start_vec)))

    for i in range(1, len(min_env)-1):
        vec_start = max(0, i - half_len)
        vec_end = min(len(min_env), i + half_len)
        vec = min_env[vec_start:vec_end]
        vec = np.array(vec)
        center_vec = np.repeat(min_env[i], len(vec))
        dist = np.sum((center_vec - vec)**2)/((1 or np.mean(vec[np.argsort(vec)[::-1]]))*len(vec))
        dists_min.append(dist)

    if(include_edges == True):
        end_vec = [(min_env[len(min_env)-1]-min_env[len(min_env)-2-i])**2  for i in range(half_len)]
        end_vec = np.array(end_vec)
        dists_min.append(np.sum(end_vec)/((1 or np.mean(end_vec[np.argsort(end_vec)[::-1]]))*len(end_vec)))

    dists_min = np.array(dists_min)
    dists_max = np.array(dists_max)
    if(plot == True):
        t_arr = np.linspace(0, len(thermal)/30, len(thermal))
        if(include_edges == True):
            lx = t_arr[lmin]
            mx = t_arr[lmax]
            plt.plot(lx, dists_min)
            plt.plot(mx, dists_max)
            plt.show()
        else:
            lx = t_arr[lmin][1:-1]
            mx = t_arr[lmax][1:-1]
            plt.plot(lx, dists_min)
            plt.plot(mx, dists_max)
            plt.show()
            
    # ld, ud = max(dists_min), max(dists_max)
    # ld_arr = []
    # for ld in dists_min:
    #     if(ld > th):
    #         ld_arr.append(ld)
    
    # ud_arr = []
    # for ud in dists_max:
    #     if(ud > th):
    #         ud_arr.append(ud)
    
    
    # print(ld)
    # print(ud)
    # window = 90

    # peaks_min = []
    # t_peaks_min = []
    # peaks_max = []
    # t_peaks_max = []
    # for i in range(len(dists_min)):
    #     if(dists_min[i] > th):
    #         peaks_min.append(i)
    #         t_peaks_min.append(t_arr[lmin][i])
    
    # for i in range(len(dists_max)):
    #     if(dists_max[i] > th):
    #         peaks.append(i)
    #         t_peaks.append(t_arr[lmax][i])
        
    # # get all points above threshold
    # # if there is more than one within 10 second span average their locations and block out  6 seconds worth
    # if((ld > th)):
    #     remove_idx = np.argmax(dists_min)
    #     lx = t_arr[lmin]
    #     cut_center = lmin[remove_idx]
    #     print(t_arr[cut_center])

    #     lower = max(0, cut_center-window)
    #     upper = min(len(thermal), cut_center+window)
    #     p1 = 1.5*thermal[0:lower]
    #     p2 = 1.5*thermal[upper:]
    #     plt.figure(figsize=(12,12))
    #     plt.plot(t_arr[0:lower], p1)
    #     plt.plot(t_arr[upper:], p2)
    #     plt.show()
    
    # if((ud > th)):
    #     remove_idx = np.argmax(dists_max)
    #     mx = t_arr[lmax]

    #     cut_center = lmax[remove_idx]
    #     print(t_arr[cut_center])
    #     lower = max(0, cut_center-window)
    #     upper = min(len(thermal), cut_center+window)
    #     p1 = 1.5*thermal[0:lower]
    #     p2 = 1.5*thermal[upper:]
    #     plt.figure(figsize=(12,12))
    #     plt.plot(t_arr[0:lower], p1)
    #     plt.plot(t_arr[upper:], p2)
    #     plt.show()
        # plt.plot(t_arr, thermal)

    return(dists_min, dists_max, lmin, lmax)

In [ ]:
np.seterr(all='raise')
for k in tqdm(range(321,322)):
    i = k
    radar_signal = radar_arr[i][:,0,0,:,2]
    radar_signal = radar_signal[:,0] + 1j*radar_signal[:,1]
    radar_signal = np.unwrap(np.angle(radar_signal))
    sig_std = np.std(radar_signal)
    radar_signal = (radar_signal - np.mean(radar_signal))
    
    dists_min, dists_max, lmin, lmax = movement_detector(radar_signal, dmin=20, dmax=20, th=20, plot=True)
    utils.plot_envelope(t_arr, radar_signal, dmin=20, dmax=20)

    center_list = find_movement_peaks(dists_min, dists_max, t_arr, th=15, x_th=150, lmin=lmin, lmax=lmax)
    half_length = 90
    print('center_list', center_list)
    signal_comps = remove_peaks(center_list, radar_signal, half_length=half_length)
    print('signal_comps len',len(signal_comps))
    pred = np.zeros(len(radar_signal))

    plt.plot(t_arr, radar_signal)
    plt.show()
    print('len', len(signal_comps))
    for sig in signal_comps:
        if(len(sig[0]) != 0):
            print('len(sig[0])', len(sig[0]), 'std(sig[0])', np.std(sig[0]))
            new_radar = (sig[0] - np.mean(sig[0]))/np.std(sig[0])
            plt.plot(t_arr[sig[1][0]:sig[1][1]], new_radar)
            plt.show()
            # print('start', sig[1][0], 'finish', sig[1][1], 'len', len(sig))
            pred[sig[1][0]:sig[1][1]] = utils.predict(new_radar , dmin=20, dmax=20, th=0.2, mode='max', plot=True)
            utils.plot_envelope(t_arr[sig[1][0]:sig[1][1]], new_radar, dmin=20, dmax=20)
            plt.show()
    if(len(signal_comps) == 0):
        pred = utils.predict(radar_signal , dmin=20, dmax=20, th=0.1, mode='max', plot=True)
    print('mean(pred)', np.mean(pred))
    # for i in range(len(center_list)):
    #     cut_center = int(center_list[i])
    #     lower = max(0, cut_center-half_length)
    #     upper = min(len(pred), cut_center+half_length)
    #     pred[lower:upper] = 0

    plt.plot(t_arr, gt_apnea[i], label='gt_radar')
    plt.plot(t_arr, pred, label='radar')
    plt.legend()
    plt.show()
    

In [ ]:
radar2_arr = []
for i in tqdm(range(1000)):
    radar_signal = radar_arr[i][:,0,0,:,2]
    radar_signal = radar_signal[:,0] + 1j*radar_signal[:,1]
    radar_signal = np.unwrap(np.angle(radar_signal))
    radar_signal = (radar_signal - np.mean(radar_signal)) / np.std(radar_signal)
    radar2_arr.append(radar_signal)

In [ ]:
7*15*100/3600

In [ ]:
th_arr = [20]#np.linspace(0.01, 4, 5)
x_th_arr = [240]
peak_th = [20]# np.linspace(5, 50, 5)
dmax_arr = [29,31]#np.linspace(20,40, 10, dtype=int)

full_arr = []
prod_arr = []

for th in tqdm(th_arr):
    for p_th in peak_th:
        for dmax in dmax_arr:
            for dmin in dmin_arr:
                for x_th in x_th_arr:

                    gt_arr = []
                    pred_arr = []


                    for i in range(1000):
                        radar_signal = radar2_arr[i]
                        dists_min, dists_max, lmin, lmax = movement_detector(radar_signal, dmin=dmin, dmax=dmax, th=p_th, plot=False)
                        # utils.plot_envelope(t_arr, radar_signal, dmin=dmin, dmax=dmax)

                        center_list = find_movement_peaks(dists_min, dists_max, t_arr, th=15, x_th=x_th, lmin=lmin, lmax=lmax)
                        half_length = 90
                        # print('center_list', center_list)
                        signal_comps = remove_peaks(center_list, radar_signal, half_length=half_length)
                        # print('signal_comps len',len(signal_comps))
                        pred = np.zeros(len(radar_signal))
                        gt = gt_apnea[i]
                        # plt.plot(t_arr, radar_signal)
                        # plt.show()
                        # print('len', len(signal_comps))
                        for sig in signal_comps:
                            if(len(sig[0]) != 0):
                                # print('len(sig[0])', len(sig[0]), 'std(sig[0])', np.std(sig[0]))
                                new_radar = (sig[0] - np.mean(sig[0]))/np.std(sig[0])
                                # plt.plot(t_arr[sig[1][0]:sig[1][1]], new_radar)
                                # plt.show()
                                # print('start', sig[1][0], 'finish', sig[1][1], 'len', len(sig))
                                pred[sig[1][0]:sig[1][1]] = utils.predict(new_radar , dmin=dmin, dmax=dmax, th=th, mode='max', plot=False)
                                # utils.plot_envelope(t_arr[sig[1][0]:sig[1][1]], new_radar, dmin=dmin, dmax=dmax)
                                # plt.show()
                        if(len(signal_comps) == 0):
                            pred = utils.predict(radar_signal , dmin=dmin, dmax=dmax, th=th, mode='max', plot=True)
                        # for i in range(len(center_list)):

                        if((np.mean(gt) > 0.1)):# or (np.mean(gt_hp) > 0.1)):
                            gt_arr.append(1)
                        else:
                            gt_arr.append(0)
                        
                        if(np.mean(pred) > 0.1):
                            pred_arr.append(1)
                        else:
                            pred_arr.append(0)
                                            
                                            # if(gt_arr[-1] != pred_arr[-1]):
                                            #     print(idx)
                                    
                                        
                    precision, recall, accuracy, confusion_matrix = utils.get_stats(np.array(pred_arr), np.array(gt_arr))
                    full_arr.append([th, dmin, dmax, x_th, precision, recall, accuracy, confusion_matrix])
                    prod_arr.append(precision*recall)
                    print(r"##################################################################\n")
                    print('th: ', th, 'dmin: ', dmin, 'dmax: ', dmax, 'x_th: ', x_th, '\n')
                    print('precision: ', precision, 'recall: ', recall, 'accuracy: ', accuracy, '\n')

In [ ]:
best_idx = np.argsort(prod_arr)[::-1]

In [ ]:
full_arr[best_idx[2]]

In [ ]:
print(precision, recall, accuracy, confusion_matrix)

In [ ]:
sig[0]

In [ ]:
id = movement_idx[0]
for i in range(3):
    for j in range(4):
        radar_signal = radar_arr[id][:,i,j,:,2]
        radar_signal = radar_signal[:,0] + 1j*radar_signal[:,1]
        radar_signal = np.unwrap(np.angle(radar_signal))
        radar_signal = (radar_signal - np.mean(radar_signal))/ np.std(radar_signal)
        plt.plot(t_arr, radar_signal, label='radar signal')
        plt.plot(t_arr, gt_radar[id], label='gt radar signal')
        plt.legend()
        plt.show()

In [ ]:
indecies = [240] #[240,241,249,250,251,253,259,295,461]
# np.random.choice(1000, 10)
# plt.plot(t_arr, thermal_arr[indecies], label='thermal')
# plt.show()

# plt.plot(gt_ir[indecies], label='gt_ir')
# Thermal 
# Radar 
for i in indecies:
    radar_signal = radar_arr[i][:,0,0,:,2]
    radar_signal = radar_signal[:,0] + 1j*radar_signal[:,1]
    radar_signal = np.unwrap(np.angle(radar_signal))
    radar_signal = (radar_signal - np.mean(radar_signal))/ np.std(radar_signal)
    print(i)
    plt.figure(figsize=(12,12))
    plt.plot(t_arr, gt_radar[i], label='gt_radar')
    plt.plot(t_arr, -radar_signal, label='radar')
    plt.plot(t_arr, gt_apnea[i], label='gt_apnea')
    plt.legend()
    plt.show()

In [ ]:
for i in tqdm(range(len(fused_dataset_train))):
    if(i in NUKS_idx):
        pass
        # print("NUKS")
    elif(i in mode_lock_idx):
        pass
        # print("Mode Lock")
    elif(i in movement_idx):
        if(i not in not_movement):
            combined_movement_idx.append(i)
  
        # print("Movement")
        # print('idx: ', i, 'std', vid_std(thermal_vid_arr[i]))
        # plt.plot(t_arr, gt_apnea[i])
        # plt.plot(t_arr, thermal_arr[i])
        # plt.show()
    else:
        # print("Normal")
        if(vid_movement_detector(thermal_vid_arr[i]) == True):
            combined_movement_idx.append(i)
        #     print('idx: ', i, 'std', vid_std(thermal_vid_arr[i]))
        #     plt.plot(t_arr, gt_apnea[i])
        #     plt.plot(t_arr, thermal_arr[i])
        #     plt.show()
    

In [ ]:
np.save(r"E:\OSA_project\Kai_data\combined_movement_idx.npy", np.array(combined_movement_idx))

In [ ]:
len(combined_movement_idx) + len(NUKS_idx) + len(mode_lock_idx)

In [ ]:
for i in tqdm(range(913,len(fused_dataset_train))):
    if((i not in combined_movement_idx) and (i not in NUKS_idx) and (i not in mode_lock_idx)):
        print('idx: ', i, 'std', vid_std(thermal_vid_arr[i]))
        plt.plot(t_arr, gt_apnea[i])
        plt.plot(t_arr, thermal_arr[i])
        plt.show()

In [ ]:
idx = 287
thermal = utils.get_thermal(fused_dataset_train[idx], order=5)
thermal = (thermal - np.mean(thermal))/np.std(thermal)
t_arr = np.linspace(0, len(thermal)/30, len(thermal))
lmin, lmax = utils.hl_envelopes_idx(thermal, dmin=30, dmax=20)
max_th = np.interp(t_arr, t_arr[lmax], thermal[lmax])
min_th = np.interp(t_arr, t_arr[lmin], thermal[lmin])
pred = ((max_th - min_th)/max(max_th - min_th) < 0.2).astype(int)
gt = fused_dataset_train[idx]["gt_apnea"][:,0]


plt.figure(figsize=(12, 12))
# pred[len(pred)-60:] = 0
utils.plot_classification_waveforms(t_arr, thermal, gt, pred, fused_dataset_train[idx]["gt_radar"][:,0])
fused_dataset_train[idx]["hypopnea"][:,0].shape, fused_dataset_train[idx]["partial_apnea"][:,0].shape

print(utils.detect_NUKS(thermal))
print(utils.detect_mode_lock(fused_dataset_train, idx))
utils.plot_envelope(t_arr, thermal, lmin, lmax)
plt.legend(loc='best')

In [ ]:
idx = 71
plt.figure(figsize=(10, 4))
plt.plot(t_arr, fused_dataset_train[idx]["gt_apnea"][:,0], color='blue', linewidth=4, label='Ground Truth Apnea Signal')
plt.plot(t_arr, fused_dataset_train[idx]["hypopnea"][:,0], label='Hypopnea')
plt.plot(t_arr, fused_dataset_train[idx]["partial_apnea"][:,0], label='Partial Apnea')
plt.legend()

In [ ]:
movement_2_idx = [
    62,
    71,
    74,
    78,
    79,
    207,
    211,
    214,
    223,
    229,
    231,
    287,
    296,
    300,
    324,
    328,
    331,
    338,
    60,
    64,
    69,
    74,
    75,
    200,
    201,
    203,
    206,
    225,
    234,
    239,
    62,
    71,
    78,
    79,
    207,
    211,
    214,
    223,
    229,
    231,
    241,
    242,
    244,
    250,
    251,
    253,
    255,
    460,
    465,
    478,
    483,
    590,
    591,
    596,
    598,
    612,
    654,
    684,
    773,
    841,
    842,
    843,
    846,
    855,
    856,
    857,
    858,
    859,
    988]

In [ ]:
acc = 0
for i in tqdm(range(len(movement_2_idx))):# tqdm(np.linspace(0,999, 1000, dtype=int)):
    idx = movement_2_idx[i]
    if(utils.movement_detector(thermal_arr[idx]) == True):
        acc = acc + 1
    elif(vid_movement_detector(thermal_vid_arr[idx]) == True):
        acc = acc + 1
print((172+acc)/(172 + len(movement_2_idx)))
print(acc/len(movement_2_idx))

In [ ]:
len(thermal_arr)

In [ ]:
movement_idx = np.load(r"E:\OSA_project\Kai_data\movement_new_idx.npy")

In [ ]:
thermal = np.mean(thermal_vid_arr[idx], axis=(-2,-1))
thermal = (thermal - np.mean(thermal))/np.std(thermal)
plt.plot(t_arr, thermal)
plt.plot(t_arr, gt_apnea[idx])

In [ ]:
idx=231

In [ ]:
for idx in movement_2_idx:
    diff_arr = np.diff(thermal_vid_arr[idx], axis=0)
    mean_arr = np.std(diff_arr**2, axis=(1,2))
    print(idx, np.std(mean_arr))

    mean_arr = mean_arr/np.std(mean_arr)
    lmin, lmax = utils.hl_envelopes_idx(mean_arr, dmin=29, dmax=31)
    utils.plot_envelope(t_arr[:-1], mean_arr, lmin, lmax)

    plt.plot(t_arr[:-1] , mean_arr)
    plt.show()
    thermal = np.mean(thermal_vid_arr[idx], axis=(-2,-1))
    thermal = (thermal - np.mean(thermal))/np.std(thermal)
    plt.plot(t_arr, thermal)
    plt.plot(t_arr, gt_apnea[idx])
    plt.show()

In [ ]:
a = np.array([[1,1], [2,3]])
print(a)
print(np.diff(a, axis=0))

In [ ]:
idx = 545
thermal = utils.get_thermal(fused_dataset_train[idx], order=5)
thermal = (thermal - np.mean(thermal))/np.std(thermal)
t_arr = np.linspace(0, len(thermal)/30, len(thermal))
plt.plot(t_arr, thermal)

In [ ]:
th

In [ ]:
np.std(thermal_vid_arr[idx]), np.std(thermal_vid_arr[504])

In [ ]:
NUKS_idx = np.load(r"E:\OSA_project\Kai_data\NUKS_idx.npy")
mode_lock_idx = np.load(r"E:\OSA_project\Kai_data\mode_lock_idx.npy")
movement_idx = np.load(r"E:\OSA_project\Kai_data\movement_idx.npy")

In [ ]:
# Th: 0.35555556, Mode: mean, Window Size: 26 --> best result so far

In [ ]:
 np.linspace(20,50, 20, dtype=int)

In [ ]:
count_apneas(dataset_apnea_train)
count_apneas(dataset_apnea_test)
len(dataset_apnea_train)

In [ ]:
full_arr = []
recall_arr = []
accuracy_arr = []
precision_arr = []
confusion_arr = []

th_arr = [0.35555556]
window_size_arr = [26]
modes = ['mean']
percentile = 0
t_arr = np.linspace(0, 1800/30, 1800)
wrong_idx = []
for th in th_arr:
    for mode in modes:
        for window_max in window_size_arr:
            for window_min in window_size_arr:
                gt_arr = []
                pred_arr = []
                

                for idx in tqdm(np.linspace(0,len(fused_dataset_train)-1,len(fused_dataset_train), dtype=int)):
                    if((idx not in NUKS_idx) and (idx not in mode_lock_idx) and ((idx not in movement_idx) or (idx in apnea_idx))):
                        # int_psd = utils.get_power_spec(thermal_arr[idx], fs=30, window_length=512, overlap=511)
                        # pred = None
                        # if(utils.no_apnea(int_psd, threshold=1) == True):
                        #     pred = np.zeros(1800)
                        # else:
                        
                        pred = utils.predict(thermal_arr[idx], dmin=window_min, dmax=30, th=th, mode=mode, percentage=percentile, plot=False)

                        gt = gt_apnea[idx]

                        if(np.mean(gt) > 0.1):
                            gt_arr.append(1)
                        else:
                            gt_arr.append(0)
                        
                        if(np.mean(pred) > 0.1):
                            pred_arr.append(1)
                        else:
                            pred_arr.append(0)
                        
                        if(pred_arr[-1] != gt_arr[-1]):
                            wrong_idx.append(idx)

                        
                        
                precision, recall, accuracy, confusion_matrix = utils.get_stats(np.array(pred_arr), np.array(gt_arr))
                recall_arr.append(recall)
                accuracy_arr.append(accuracy)
                precision_arr.append(precision)
                confusion_arr.append(confusion_matrix)
                full_arr.append((th, mode, window_size, recall, precision, accuracy, confusion_matrix))

                # print(f"Th: {th}, Mode: {mode}, Window Size: {window_size}")
                # print(f"Recall: {recall}, Precision: {precision}, Accuracy: {accuracy}")
                # print(confusion_matrix)


In [ ]:
recall_arr[0], precision_arr[0], accuracy_arr[0], confusion_arr[0]

In [ ]:
recall_arr[0], precision_arr[0], accuracy_arr[0], confusion_arr[0]

In [ ]:
k=2
thermal = thermal_arr[wrong_idx[k]]
t_arr = np.linspace(0, len(thermal)/30, len(thermal))
lmin, lmax = utils.hl_envelopes_idx(thermal, dmin=30, dmax=20)
max_th = np.interp(t_arr, t_arr[lmax], thermal[lmax])
min_th = np.interp(t_arr, t_arr[lmin], thermal[lmin])
pred = ((max_th - min_th)/max(max_th - min_th) < 0.2).astype(int)
gt = gt_apnea[wrong_idx[k]]
int_psd = utils.get_power_spec(thermal, fs=30, window_length=512, overlap=511)

plt.figure(figsize=(12, 12))
# pred[len(pred)-60:] = 0
# utils.plot_classification_waveforms(t_arr, thermal, gt, pred, fused_dataset_train[idx]["gt_radar"][:,0])
# fused_dataset_train[idx]["hypopnea"][:,0].shape, fused_dataset_train[idx]["partial_apnea"][:,0].shape
plt.plot(t_arr, gt, color='blue', linewidth=4, label='Ground Truth Apnea Signal')
plt.plot(t_arr, thermal, label='Predicted Apnea Signal')
processed = utils.lowpass_filter(thermal, fc=0.3, fs=30, order=5)
plt.plot(t_arr, processed, label='Low Pass Filtered Signal')
                                  
utils.plot_envelope(t_arr, thermal, lmin, lmax)
plt.legend(loc='best')

In [ ]:
plt.plot(t_arr, processed, label='Low Pass Filtered Signal')
plt.plot(t_arr, thermal)
plt.legend()

In [ ]:
plt.plot(t_arr[255:-256], int_psd)
plt.plot(t_arr, gt)

In [ ]:
prod = np.array(precision_arr)*np.array(recall_arr)

# if array contains nan set that entry to zero
for i in range(len(prod)):
    if(math.isnan(prod[i])):
        prod[i] = 0

In [ ]:
best_idx = np.argsort(prod)[::-1]

In [ ]:
precision_arr[best_idx[0]], recall_arr[best_idx[0]], accuracy_arr[best_idx[0]], confusion_arr[best_idx[0]]

In [ ]:
np.save(r"E:\OSA_project\Emir_Clean\th_arr.npy", th_arr)
np.save(r"E:\OSA_project\Emir_Clean\window_size_arr.npy", window_size_arr)
np.save(r"E:\OSA_project\Emir_Clean\modes.npy", modes)
np.save(r"E:\OSA_project\Emir_Clean\recall_arr.npy", recall_arr)
np.save(r"E:\OSA_project\Emir_Clean\accuracy_arr.npy", accuracy_arr)
np.save(r"E:\OSA_project\Emir_Clean\precision_arr.npy", precision_arr)
np.save(r"E:\OSA_project\Emir_Clean\confusion_arr.npy", confusion_arr)

In [ ]:
plt.plot(precision_arr)
plt.show()
plt.plot(recall_arr)
plt.show()

In [ ]:
# 2.0) come up with hypopnea model
# 2.1) implement AHI and other relevant metrics
# 2.2) explain model shortcomings